<a href="https://colab.research.google.com/github/SrushtiGunjal/whatsapp_bots/blob/main/Copy_of_QWEN_0_5B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall -y transformers
!pip install transformers==4.38.2
!pip install accelerate einops

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import pandas as pd
from google.colab import files

# 1️⃣ Load Qwen1.5-0.5B model
model_id = "Qwen/Qwen1.5-0.5B-Chat"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    torch_dtype=torch.float16,
    device_map="auto"
)

# 2️⃣ Upload the CSV
uploaded = files.upload()
csv_path = list(uploaded.keys())[0]
df = pd.read_csv(csv_path)

# 3️⃣ Convert CSV to readable format (limit to 4000 characters)
csv_text = ""
for _, row in df.iterrows():
    row_text = ", ".join(f"{col}: {val}" for col, val in row.items())
    csv_text += row_text + "\n"
csv_text = csv_text[:4000]  # Avoid exceeding token limit

# 4️⃣ Define your question
query = "what is the status and remark of Aarav Salt & Chemicals Pvt. Ltd. with turbine code M121?"

# 5️⃣ Create chat messages
messages = [
    {"role": "system", "content": "You are a helpful assistant. Use the provided data to answer the user's question."},
    {"role": "user", "content": f"Here is the data:\n{csv_text}"},
    {"role": "user", "content": query}
]

# 6️⃣ Generate response
input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt").to(model.device)
output = model.generate(input_ids, max_new_tokens=200, temperature=0.7, do_sample=True)
response = tokenizer.decode(output[0][input_ids.shape[-1]:], skip_special_tokens=True)

# 7️⃣ Output result
print(f"User: {query}\nBot: {response}")


Found existing installation: transformers 4.38.2
Uninstalling transformers-4.38.2:
  Successfully uninstalled transformers-4.38.2
  Using cached transformers-4.38.2-py3-none-any.whl.metadata (130 kB)
Using cached transformers-4.38.2-py3-none-any.whl (8.5 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.38.2 which is incompatible.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Saving fact_dpos_1_day.csv to fact_dpos_1_day (1).csv
User: what is the status and remark of Aarav Salt & Chemicals Pvt. Ltd. with turbine code M121?
Bot: The status of Aarav Salt & Chemicals Pvt. Ltd. with turbine code M121 is "Available".


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import pandas as pd

# 1️⃣ Load model and tokenizer
model_id = "Qwen/Qwen1.5-0.5B-Chat"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
).to("cuda" if torch.cuda.is_available() else "cpu")

# 2️⃣ Upload and read the CSV file
from google.colab import files
uploaded = files.upload()

# Auto-extract the filename from uploaded dict
csv_path = next(iter(uploaded))
try:
    df = pd.read_csv(csv_path)
except Exception as e:
    print(f"Error reading CSV: {e}")
    raise

# 3️⃣ Convert CSV data to readable format (safe truncation)
def dataframe_to_text(dataframe, max_chars=3500):
    lines = []
    for _, row in dataframe.iterrows():
        row_text = ", ".join(f"{col}: {val}" for col, val in row.items())
        lines.append(row_text)
        if sum(len(line) for line in lines) > max_chars:
            break
    return "\n".join(lines)

csv_text = dataframe_to_text(df)

# 4️⃣ Define your natural language query
query = "What is the status and remark of Aarav Salt & Chemicals Pvt. Ltd. with turbine code M121?"

# 5️⃣ Prepare messages for Qwen chat model
messages = [
    {"role": "system", "content": "You are a helpful assistant. Use the provided CSV data to answer the user's question accurately."},
    {"role": "user", "content": f"The data is:\n{csv_text}"},
    {"role": "user", "content": query}
]

# 6️⃣ Generate response
with torch.no_grad():
    input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt").to(model.device)
    output = model.generate(input_ids, max_new_tokens=200, do_sample=False, temperature=0.7)
    response = tokenizer.decode(output[0][input_ids.shape[-1]:], skip_special_tokens=True)

# 7️⃣ Show the result
print(f"User: {query}\n\nBot: {response}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Saving fact_dpos_1_day.csv to fact_dpos_1_day (2).csv


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


User: What is the status and remark of Aarav Salt & Chemicals Pvt. Ltd. with turbine code M121?

Bot: The status of Aarav Salt & Chemicals Pvt. Ltd. with turbine code M121 is "Available". The remark is "Turbine was under breakdown due to Elec Yaw Sensor Error stop from 08:25 hours and restored at 10:21 hours."
